In [2]:
import sys
path = "C:\\Users\\Paul Valcke\\Documents\\GitHub\\GEMMES" 
sys.path.insert(0, path)
import pygemmes as pgm


#####################################################################################
 _______            ______  ________  ____    ____  ____    ____  ________   ______    
|_   __ \         .' ___  ||_   __  ||_   \  /   _||_   \  /   _||_   __  |.' ____  \   
  | |__) |_   __ / .'   \_|  | |_ \_|  |   \/   |    |   \/   |    | |_ \_|| (___ \_|  
  |  ___/[ \ [  ]| |   ____  |  _| _   | |\  /| |    | |\  /| |    |  _| _  _.____`.   
 _| |_    \ '/ / \ `.___]  |_| |__/ | _| |_\/_| |_  _| |_\/_| |_  _| |__/ || \____) |  
|_____| [\_:  /   `._____.'|________||_____||_____||_____||_____||________| \______.'  
         \__.'                           
#####################################################################################                                                                                                                                   

Welcome in Pygemmes, a modular library to prototype and study dynamical systems !
This library is oriented toward generation

## CREATING A MODEL THROUGH AN INTERFACE


In [3]:
import ipywidgets as widgets
from IPython.display import display,HTML
from IPython.display import clear_output
import numpy as np
import pandas as pd
style = {'description_width': 'initial'}

%matplotlib widget
pd.set_option('display.max_colwidth', None)
default = 'GK'
AllModels = pgm.get_available_models(details=True)


def pprint(ldf):
    try :
        ldf = ldf.style.set_properties(**{'text-align': 'left'})
    except BaseException:
        pass
    return display(HTML(ldf.to_html().replace("\\n","<br>")))

In [3]:
AllModels.loc['GK']

Folder                                                                                                                                                                                                                                                                                                  Extended
Preset                                                                                                                                                                                                                                                                              [default, farfromEQ, crisis]
Documentation    \nDESCRIPTION : This is a Goodwin model with possibility to get loans at the bank, and have a price dynamics.\n              As described by keen + inflation\n\nTYPICAL BEHAVIOR : Convergence toward solow point ( good equilibrium) or debt crisis\nLINKTOARTICLE:\n\n@author: Paul Valcke\n
Name: GK, dtype: object

In [4]:

hub=pgm.Hub('GK',verb=False)
hub.run(verb=False)

In [5]:

#### ALL BUTTONS 


# %% TAB "Show All Models" ################################
outPGM = widgets.Output()

ShowModels = widgets.Button(description='Show All Models')
def ShowModel_event(obj):
    with outPGM:
        outPGM.clear_output()
        pprint(pgm.get_available_models())
ShowModels.on_click(ShowModel_event)

ShowFields = widgets.Button(description='Show All Fields')
def ShowFields_event(obj):
    with outPGM:
        outPGM.clear_output()
        pprint(pgm.get_available_fields())
ShowFields.on_click(ShowFields_event)

ShowPlots = widgets.Button(description='Show All Plots')
def ShowPlots_event(obj):
    with outPGM:
        outPGM.clear_output()
        pprint(pgm.get_available_plots())
ShowPlots.on_click(ShowPlots_event)

show_widgets0= widgets.HBox([ShowModels,ShowFields,ShowPlots])
show_widgets= widgets.VBox([show_widgets0,outPGM])

# TAB "Load model" #######################################
dropdown = widgets.Dropdown(options=list(AllModels.index),value=default,description='Model file :')
presets = widgets.Dropdown(options=[None]+AllModels.loc['Goodwin'].loc['Preset'], description='Preset :')
def valuechange(change):
    presets.options = [None]+AllModels.loc[change['new']].loc['Preset']
    with outLoad:
        outLoad.clear_output()
        display(AllModels.loc[change['new']].loc['Documentation'])
        #display(HTML(.replace("\\n","<br>")))
dropdown.observe(valuechange, names='value')

# Verbatim 
verbatim = widgets.Dropdown(options=[True,False],value=True,description='Verbose')

# Create the hub
createhub = widgets.Button(description='LOAD !')
def createhub_event(obj):
    with outLoad:
        outLoad.clear_output()
        hub= pgm.Hub(dropdown.value,preset=presets.value,verb=verbatim.value)
    return hub
createhub.on_click(createhub_event)

outLoad = widgets.Output()

# Verbatim checkbox 
input_widgets0= widgets.HBox([dropdown,presets,verbatim,createhub])
input_widgets= widgets.VBox([input_widgets0,outLoad])

# TAB "Network" #########################################


createNetwork = widgets.Button(description='Create Network')
outNetwork= widgets.Output()

dropdownP = widgets.Dropdown(options=[True,False],value=True,description='Show parameters :',style=style)
dropdownA = widgets.Dropdown(options=[True,False],value=True,description='Show Auxilliary :',style=style)

def createNetwork_event(obj):
    with outNetwork:
        outNetwork.clear_output()
        f= hub.get_Network(params=dropdownP.value,auxilliary=dropdownA)  
        display(HTML(f.read()))
    return hub
createNetwork.on_click(createNetwork_event)
net0 = widgets.HBox([dropdownP,dropdownA])
network_widget = widgets.VBox([net0,createNetwork,outNetwork])
                  # state,differential,parameters
#hub.get_Network(auxilliary=False,params=True)   # remove auxilliary statevar and differential
#hub.get_Network(filters=('Pi',))                # remove the variable Pi and its connexions
#hub.get_Network(filters=('Pi',),redirect=True)  # all connexions from Pi are reconnected

# TAB "execute" #########################################

### CHANGE VARIABLE 
# Choose variable
if hub is not None: 
    Allvars = list(set(hub.dmisc['dfunc_order']['parameter']
                    +hub.dmisc['dfunc_order']['parameters']
                    +hub.dmisc['dfunc_order']['differential'])
                    -set(['__ONE__','dt']))
else :
    Allvars=['']
vardrop = widgets.Dropdown(options=Allvars,value=Allvars[0],description='Field to change :',style=style)

# Choose value
setvalue =  widgets.FloatText(
    value=0,
    description='Set value:')

#  Set  
clickset = widgets.Button(description='Set !')
def clickset_event(obj):
    hub.set_dparam(vardrop.value,float(setvalue.value))
clickset.on_click(clickset_event)

run_widgets_0 = widgets.HBox([vardrop,setvalue,clickset])

### RUN 
clickrun = widgets.Button(description='RUN!')
def clickrun_event(obj):
    hub.run()
clickrun.on_click(clickrun_event)
run_widgets_1 = widgets.HBox([clickrun])


### REINTERPOLATE 
# Choose value
setReinterp =  widgets.FloatText(
    value=100,
    description='Number of temporal points:',style=style)

#  Set  
clickset = widgets.Button(description='Set !')
def clickset_event(obj):
    hub.reinterpolate_dparam(setReinterp.value)
clickset.on_click(clickset_event)

run_widgets_2 = widgets.HBox([setReinterp,clickset])

run_widgets = widgets.VBox([run_widgets_0,run_widgets_1,run_widgets_2])



# TAB "instrospection" ##################################
outINTROS = widgets.Output()

clicksummary = widgets.Button(description='get summary')
def clicksummary_event(obj):
    with outINTROS:
        
        outINTROS.clear_output()
        hub.get_summary()
clicksummary.on_click(clicksummary_event)

Categories = ['eqtype','source_exp','definition','com','group','units','symbol','isneeded']
dropdownCAT = widgets.Dropdown(options=Categories,value='eqtype',description='Sorting :')

clickFields =widgets.Button(description='get Fields')
def clickFields_event(obj):
    with outINTROS:
        outINTROS.clear_output()
        R=hub.get_dparam()
        Rpandas= {k0:{k:R[k0][k] for k,v in R[k0].items() if k in Categories} for k0 in R.keys()}
        AllFields=pd.DataFrame(Rpandas,index=Categories).transpose().sort_values(by=[dropdownCAT.value])
        display(AllFields)
clickFields.on_click(clickFields_event)


introspect_widgets = widgets.VBox([dropdownCAT,clickFields,outINTROS])

# TAB "plots" ###########################################
outPLOT = widgets.Output()

R=hub.get_dparam()
t=R['time']['value'][:,0,0,0,0]
TimeSlider = widgets.FloatRangeSlider(
    value=[t[0], t[-1]],
    min=t[0],
    max=t[-1],
    step=t[1]-t[0],
    description='Time range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

Parrallel = widgets.Dropdown(
    options=np.arange(hub.dparam['nx']['value']),
    value=0,
    description='parrallel :',
    disabled=False
)

Region = widgets.Dropdown(
    options=hub.dparam['nr']['list'],
    value=hub.dparam['nr']['list'][0],
    description='Region :',
    disabled=False
) 


Plotvars = hub.dmisc['dfunc_order']['statevar']+hub.dmisc['dfunc_order']['differential']

varXdrop = widgets.Dropdown(options=Plotvars,value='time',description='X axis :')
varYdrop = widgets.Dropdown(options=Plotvars,value='time',description='Y axis :')
varCdrop = widgets.Dropdown(options=Plotvars,value='time',description='color :')
plot_widgets0 = widgets.VBox([TimeSlider,Parrallel,Region])

clickplot = widgets.Button(description='Plot !')
def clickplot_event(obj):
    with outPLOT:
        pgm.plots.phasespace(hub,x=varXdrop.value,
                         y=varYdrop.value,
                         color = varCdrop.value,idx=0,Region=0)
clickplot.on_click(clickplot_event)
plot_widgets1 = widgets.HBox([varXdrop,varYdrop,varCdrop,clickplot])

plot_widgets= widgets.VBox([plot_widgets0,plot_widgets1,outPLOT])


# TAB "saveload" ########################################




############################ CREATE THE INTERFACE ################################
##################################################################################

#plot_widgets = widgets.HBox([])network_widget

#plot_widgets = widgets.HBox([])network_widget
saveload_widgets = widgets.HBox([])

tab = widgets.Tab()
tab.children= [show_widgets,input_widgets,network_widget,run_widgets,introspect_widgets,plot_widgets]
tab.titles = ['Exploring Pygemmes','Loading model','Network','Execute','Introspection','Plots']

display(tab)



In [ ]:
hub.get_summary()

In [ ]:
accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
display(accordion)


In [ ]:
from pyvis import network as net 
import networkx as nx 
g=net.Network(notebook=True)
nxg = nx.complete_graph(5)
g.from_nx(nxg)
g.show("example.html")